In [1]:
from evaluation import compute_similarity, auc
from loss import pairwise_loss, triplet_loss
from gmn_utils import *
from configure import *
import numpy as np
import torch.nn as nn
import collections
import time
import os
from datetime import datetime
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch
from torch_geometric.nn import dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GATWithMinCutPooling(torch.nn.Module):
    def __init__(
        self,
        num_features,
        num_classes,
        num_hidden=8,
        heads=8,
        dropout=0.6,
        name="GAT",
    ):  
        super(GATWithMinCutPooling, self).__init__()
        self.name = name

        self.conv1 = GATConv(
            in_channels=num_features,
            out_channels=num_hidden,
            heads=heads,
            dropout=dropout,
        )

        self.conv2 = GATConv(
            in_channels=num_hidden * heads,
            out_channels=num_classes,
            heads=1,
            dropout=dropout,
        )
        
        self.num_classes = num_classes
        self.feature_transform = torch.nn.Linear(num_classes, num_features)
        
    def forward(self, data):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = self.conv1(x, data.edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index)
        
        x_dense, mask = to_dense_batch(data.x, data.batch)
        adj_dense = to_dense_adj(data.edge_index, data.batch)

        num_clusters = dataset.num_classes

        transform = torch.nn.Linear(x.size(1), num_clusters)
        s = transform(x)
        s = torch.softmax(s, dim=1)

        x_pool, adj_pool, mincut_loss, ortho_loss = dense_mincut_pool(x_dense, adj_dense, s, mask=mask)
        x_pool = x_pool.squeeze(0)

        return x, x_pool, adj_pool, mincut_loss, ortho_loss

classification_criterion = torch.nn.CrossEntropyLoss()

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
    classification_loss = classification_criterion(x_class[data.train_mask], data.y[data.train_mask])
    loss = classification_loss + mincut_loss + ortho_loss
    loss.backward()
    optimizer.step()
    return loss.item(), classification_loss, mincut_loss, ortho_loss

def validate(model, data):
    model.eval()
    with torch.no_grad():
        x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
        classification_loss = classification_criterion(x_class[data.val_mask], data.y[data.val_mask])
        loss = classification_loss + mincut_loss + ortho_loss
        return loss.item(), classification_loss, mincut_loss, ortho_loss

def test(model, data):
    model.eval()
    with torch.no_grad():
        x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
        classification_loss = classification_criterion(x_class[data.test_mask], data.y[data.test_mask])
        loss = classification_loss + mincut_loss + ortho_loss
        return loss.item(), classification_loss, mincut_loss, ortho_loss

dataset = Planetoid(root='../Cora', name='Cora')
data = dataset[0]

classification_criterion = torch.nn.CrossEntropyLoss()


def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
    classification_loss = classification_criterion(x_class[data.train_mask], data.y[data.train_mask])
    loss = classification_loss + mincut_loss + ortho_loss
    loss.backward()
    optimizer.step()
    return loss.item(), classification_loss, mincut_loss, ortho_loss

def validate(model, data):
    model.eval()
    with torch.no_grad():
        x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
        classification_loss = classification_criterion(x_class[data.val_mask], data.y[data.val_mask])
        loss = classification_loss + mincut_loss + ortho_loss
        return loss.item(), classification_loss, mincut_loss, ortho_loss

def test(model, data):
    model.eval()
    with torch.no_grad():
        x_class, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)
        classification_loss = classification_criterion(x_class[data.test_mask], data.y[data.test_mask])
        loss = classification_loss + mincut_loss + ortho_loss
        return loss.item(), classification_loss, mincut_loss, ortho_loss

dataset = Planetoid(root='../Cora', name='Cora')
data = dataset[0]

In [3]:
final_model = GATWithMinCutPooling(num_features=dataset.num_features, num_classes=dataset.num_classes, dropout=0.6)
optimizer = torch.optim.Adam(final_model.parameters(), lr=0.005)    

for epoch in range(50):
    train_loss, train_class_loss, train_mincut_loss, train_ortho_loss = train(final_model, data, optimizer)
             
test_loss, test_class_loss, test_mincut_loss, test_ortho_loss = test(final_model, data)
print(f"Test Loss: {test_loss}")
print(f'\ttest_class_loss = {test_class_loss}, test_mincut_loss = {test_mincut_loss}, test_ortho_loss = {test_ortho_loss}') 


Test Loss: 0.7703790068626404
	test_class_loss = 0.6601805686950684, test_mincut_loss = -0.9986997246742249, test_ortho_loss = 1.1088981628417969


In [4]:
model = final_model
model.eval()
with torch.no_grad():
    out, x_pool, adj_pool, mincut_loss, ortho_loss = model(data)

edge_index_pool = dense_to_sparse(adj_pool)[0]

pooled_data = Data(x=x_pool, edge_index=edge_index_pool)

In [5]:
g1 = Data(x=data.x, edge_index=data.edge_index)
g2 = pooled_data

In [6]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Print configure
config = get_default_config()
for k, v in config.items():
    print("%s= %s" % (k, v))

# Set random seeds
seed = config["seed"]
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)


training_set, validation_set = build_datasets(config)

if config["training"]["mode"] == "pair":
    training_data_iter = training_set.pairs(config["training"]["batch_size"])
    first_batch_graphs, _ = next(training_data_iter)
else:
    training_data_iter = training_set.triplets(config["training"]["batch_size"])
    first_batch_graphs = next(training_data_iter)

node_feature_dim = first_batch_graphs.node_features.shape[-1]
edge_feature_dim = first_batch_graphs.edge_features.shape[-1]

model, optimizer = build_model(config, node_feature_dim, edge_feature_dim)
model.to(device)

encoder= {'node_hidden_sizes': [32], 'node_feature_dim': 1, 'edge_hidden_sizes': [16]}
aggregator= {'node_hidden_sizes': [128], 'graph_transform_sizes': [128], 'input_size': [32], 'gated': True, 'aggregation_type': 'sum'}
graph_embedding_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching'}
graph_matching_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching', 'similarity': 'dotproduct'}
model_type= matching
data= {'problem': 'graph_edit_distance', 'datase

GraphMatchingNet(
  (_encoder): GraphEncoder(
    (MLP1): Sequential(
      (0): Linear(in_features=8, out_features=32, bias=True)
    )
    (MLP2): Sequential(
      (0): Linear(in_features=4, out_features=16, bias=True)
    )
  )
  (_aggregator): GraphAggregator(
    (MLP1): Sequential(
      (0): Linear(in_features=32, out_features=256, bias=True)
    )
    (MLP2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (_prop_layers): ModuleList(
    (0-4): 5 x GraphPropMatchingLayer(
      (_message_net): Sequential(
        (0): Linear(in_features=80, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
      (_reverse_message_net): Sequential(
        (0): Linear(in_features=80, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
      (GRU): GRU(96, 32)
    )
  )
)

In [7]:
import numpy as np
from sklearn.decomposition import PCA
import torch

pca = PCA(n_components=8)
reduced_node_features = pca.fit_transform(g1.x)

from_idx = g1.edge_index[0]
to_idx = g1.edge_index[1]

num_edges = g1.edge_index.shape[1]

graph_idx = np.zeros(g1.x.shape[0], dtype=int)

node_features = torch.tensor(reduced_node_features, dtype=torch.float32)
edge_features = torch.ones((num_edges, 4), dtype=torch.float32)
from_idx = torch.tensor(from_idx, dtype=torch.long)
to_idx = torch.tensor(to_idx, dtype=torch.long)
graph_idx = torch.tensor(graph_idx, dtype=torch.long)

model.eval()

stacked_node_features = torch.cat((node_features, node_features.clone()))
stacked_edge_features = torch.cat((edge_features, edge_features.clone()))

stacked_from_idx = torch.cat((from_idx, from_idx.clone()+g1.x.shape[0]))
stacked_to_idx = torch.cat((to_idx, to_idx.clone()+g1.x.shape[0]))

stacked_graph_idx = torch.cat((torch.tensor(np.zeros(g1.x.shape[0], dtype=int)), torch.tensor(np.ones(g1.x.shape[0], dtype=int))))

graph_vectors = model(
        stacked_node_features.to(device),
        stacked_edge_features.to(device),
        stacked_from_idx.to(device), 
        stacked_to_idx.to(device),
        stacked_graph_idx.to(device),
        2
    )

x, y = reshape_and_split_tensor(graph_vectors, 2)
similarity = compute_similarity(config, x, y)
print(similarity)

/var/folders/st/cyh3v8ln22sdswp1s386w5vw0000gn/T/ipykernel_53819/3761369209.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from_idx = torch.tensor(from_idx, dtype=torch.long)
/var/folders/st/cyh3v8ln22sdswp1s386w5vw0000gn/T/ipykernel_53819/3761369209.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  to_idx = torch.tensor(to_idx, dtype=torch.long)


tensor([1.0000], grad_fn=<SumBackward1>)
